<a href="https://colab.research.google.com/github/Crysflair/ml-mnist-supervised-unsupervised/blob/master/CNN%2BMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CNN + MNIST for AI course work (2018, BIT)

This is a practice of tensorflow and tensorboard.  

Note there is still something wrong with the code: tensorboard fails to show many curves as I want. 

It might dues to namespace's issue?

In [0]:
data_dir = "./MO_MNIST_data"

In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data  # download and extract the data set automatically


#### DEFINE FREQUENTLY USED FUNCTIONS ####

# initialize every weight matrix as truncated normal for relu.
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

# initialize every bias vector as a constant.
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


In [0]:
def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('mo_auto_summaries'):
      mean = tf.reduce_mean(var)
      tf.summary.scalar('mean', mean)
      with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        
      # 使用tf.summary.scaler记录记录下标准差，最大值，最小值
      tf.summary.scalar('stddev', stddev)
      tf.summary.scalar('max', tf.reduce_max(var))
      tf.summary.scalar('min', tf.reduce_min(var))
      
      # 用直方图记录参数的分布
      tf.summary.histogram('histogram', var)
      
# reference: https://blog.csdn.net/sinat_33761963/article/details/62433234 

In [0]:
#### CONSTRUCT THE MODEL ####
# reference: https://www.cnblogs.com/Ran-Chen/p/9220739.html

tf.reset_default_graph()

# get the data source
mnist = input_data.read_data_sets(data_dir, one_hot=True);

# input image:pixel 28*28 = 784
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784])
    y_ = tf.placeholder('float', [None, 10])  # y_ is realistic result

with tf.name_scope('image'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])  # any dim, width, height, channel(depth)
    tf.summary.image('input_image', x_image, 8)


# the first convolution layer
with tf.name_scope('conv_layer1'):
    W_conv1 = weight_variable([5, 5, 1, 32])  # convolution kernel: 5*5*1, number of kernel: 32
    variable_summaries(W_conv1)
    b_conv1 = bias_variable([32])
    # variable_summaries(b_conv1)
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)  # make convolution, output: 28*28*32

with tf.name_scope('pooling_layer'):
    h_pool1 = max_pool_2x2(h_conv1)  # make pooling, output: 14*14*32

# the second convolution layer
with tf.name_scope('conv_layer2'):
    W_conv2 = weight_variable([5, 5, 32, 64])  # convolution kernel: 5*5, depth: 32, number of kernel: 64
    b_conv2 = bias_variable([64])
    variable_summaries(W_conv2)
    # variable_summaries(b_conv2)
    
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)  # output: 14*14*64

with tf.name_scope('pooling_layer'):
    h_pool2 = max_pool_2x2(h_conv2)  # output: 7*7*64

    
# the first fully connected layer
with tf.name_scope('fc_layer3'):
    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    b_fc1 = bias_variable([1024])  # size: 1*1024
    variable_summaries(W_fc1)
    # variable_summaries(b_fc1)
    
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)  # output: 1*1024

# dropout
with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# the second fully connected layer
with tf.name_scope('output_fc_layer4'):
    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])  # size: 1*10
    variable_summaries(W_fc2)
    # variable_summaries(b_fc2)

with tf.name_scope('softmax'):
    y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)  # output: 1*10

with tf.name_scope('lost'):
    cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
    tf.summary.scalar('lost', cross_entropy)

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    tf.summary.scalar('accuracy', accuracy)

In [0]:
#### PREPAREMENT BEFORE TRAINING ####

# merge everything
merged = tf.summary.merge_all()

# init all variables
init = tf.global_variables_initializer()


In [6]:
import tensorboardcolab
tbc = tensorboardcolab.TensorBoardColab()
train_summary = tbc.get_writer()

Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
http://dbe4ac49.ngrok.io


In [0]:
# run session
with tf.Session() as sess:
  
    train_summary.add_graph(sess.graph)
    
    sess.run(init)
    for i in range(2000):  # train 2000 times
        batch = mnist.train.next_batch(50)
        result, _ = sess.run([merged, train_step], feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
       
        if i % 20 == 0:
            # print train accuracy
            train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})  # no dropout
            print('step %d, training accuracy %g' % (i, train_accuracy))

            # summary
            train_summary.add_summary(result, i)
    
    # close summary to free resources
    train_summary.close() 
    print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
